# ANÁLISIS DEL CONJUNTO DE DATOS Y EVALUACIÓN DEL MODELO

## 1. Conjuntos de datos

En este apartado, se describen los diferentes conjuntos de datos obtenidos en base a los diálogos. Cada registro está formado por una columna de entrada, que es el texto a clasificar, y una columna de salida, que es la intención en la que se clasifica el texto.

Según se define en el apartado **4. Identificación de requisitos** de la memoria, se van a tratar las siguientes intenciones:

- Registrar saludo
- Registrar toma de medicamento
- Registrar estado emocional
- Registrar cita médica
- Registrar despedida

Los objetivos cubiertos son:
- Detectar las necesidades de las personas mayores: en base al conjunto de datos etiquetado, se listan las necesidades y el número de registros que se han clasificado en cada texto. De este modo, es posible evaluar qué necesidades son las más importantes.
- Conseguir un conjunto de datos de un mínimo de 100 registros: El objetivo es conseguir un conjunto de datos balanceado y sin duplicados de un mínimo de 100 registros de las intenciones que se van a tratar. Utilizamos sólo las intenciones que tienen más registros.
- Conseguir accuracy 85%: lala

In [30]:
import json
import matplotlib.pyplot as plt
import pandas as pd

def getDataset(version, sufix=""):
    path = f"/home/jovyan/data/dataset/{version}/data{sufix}.json"
    with open(path, 'r') as f:
      data = json.load(f)
    return data

def getDatasetAsDictionaryText(version):
    dataset = getDataset(version)
    datasetDictionary = {}
    for item in dataset:
        datasetDictionary[item["text"]] = item
    return datasetDictionary

def dataToCSV(dataset, version, sufix=""):
    entries = []
    for entry in dataset:
        text = entry["text"]
        intent = entry["intent"]
        entryText = f"{text};{intent}"
        entries.append(entryText)
    path = f"/home/jovyan/data/dataset/{version}/data{sufix}.csv"
    with open(path,'w') as f:
        for line in entries:
            f.write(line)
            f.write('\n')
    
def writeCSV(entries, version, sufix=""):
    path = f"/home/jovyan/data/dataset/{version}/data{sufix}.csv"
    with open(path, 'w') as f:
      json.dump(entries, f)

def loadDataframe(version, sufix=""):
    dataset = getDataset(version, sufix)
    entries = dataToCSV(dataset, version, sufix)
    path = f"/home/jovyan/data/dataset/{version}/data{sufix}.csv"
    df = pd.read_csv(path, names=["text", "intent"], sep=";")
    return df

def filterDataframe(df):
    filteredColumns = ['REGISTRAR_TOMA_MEDICAMENTO', 'REGISTRAR_ESTADO_EMOCIONAL', 'REGISTRAR_CITA_MEDICA', 'REGISTRAR_SALUDO', 'REGISTRAR_DESPEDIDA']
    filter = df['intent'].isin(filteredColumns)
    dfFiltered = df[filter]
    return dfFiltered

def saveTrainTestDataset(version):
    dataset = getDatasetAsDictionaryText(version)
    df = loadDataframe(version)
    train=df.sample(frac=0.8,random_state=2) #random state is a seed value
    test=df.drop(train.index)
    saveDF(version, train, "train", dataset)
    saveDF(version, test, "test", dataset)
    dfTrain = loadDataframe(DATASET_VERSION, "_train")
    dfTest = loadDataframe(DATASET_VERSION, "_test")
    return dfTrain, dfTest
        
def saveDF(version, df, split, dataset):
    output = {}
    for index, row in df.iterrows():
        output[row["text"]] = dataset[row["text"]]

    toJSON = []
               
    for item in output.keys():
        toJSON.append(output[item])
    

    path = f"/home/jovyan/data/dataset/{DATASET_VERSION}/data_{split}.json"
    with open(path, "w", encoding="utf-8") as f:
        json.dump(toJSON, f, ensure_ascii=False, indent=4)
        print(f"Saved at {path}")

In [31]:
DATASETS = {}
DATATRAIN = {}
DATATEST = {}

### 1.1. Conjunto v1

In [32]:
DATASET_VERSION = "v1" # Versión del conjunto de datos
df = loadDataframe(DATASET_VERSION)
df.head()

,text,intent
0,"Espero que no se enamore de ti como en Her, au...",NO_IDENTIFICADO
1,Hola estoy muy sola necesito de tus consejos y...,REGISTRAR_ESTADO_EMOCIONAL
2,Este mismo mes de junio a las 8'30 con el enfe...,REGISTRAR_CITA_MEDICA
3,"Hola, he estado un poco ocupado, lo siento, in...",REGISTRAR_ESTADO_EMOCIONAL
4,el rinialer me toca a las 9h,REGISTRAR_TOMA_MEDICAMENTO


In [33]:
df.shape

(176, 2)

In [34]:
df["intent"].unique()

array(['NO_IDENTIFICADO', 'REGISTRAR_ESTADO_EMOCIONAL',
       'REGISTRAR_CITA_MEDICA', 'REGISTRAR_TOMA_MEDICAMENTO',
       'REGISTRAR_NECESIDAD', 'REGISTRAR_DESPEDIDA', 'REGISTRAR_SALUDO',
       'REGISTRAR_ALABANZA', 'REGISTRAR_CONFIRMACION',
       'CONSULTAR_ESTADO_PERSONA_MAYOR', 'REGISTRAR_COVID',
       'REGISTRAR_SITUACION_ADVERSA', 'ANOTAR_TOMA_MEDICAMENTO',
       'ACTUALIZAR_TOMA_MEDICAMENTO', 'REGISTRAR_MEDIDA_MEDICA',
       'REGISTRAR_SINTOMA', 'REGISTRAR_ANECDOTA', 'REGISTRAR_DUDA',
       'REGISTRAR_DOMOTICA', 'RECORDAR_MEDICACION',
       'RECORDAR_CITA_MEDICA', 'REGISTRAR_ACTIVIDAD', 'CONSULTAR_TIEMPO',
       'REGISTRAR_TELEGRAM', 'REGISTRAR_RECHAZO', 'CONSULTAR_NUTRICION'],
      dtype=object)

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176 entries, 0 to 175
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    176 non-null    object
 1   intent  176 non-null    object
dtypes: object(2)
memory usage: 2.9+ KB


In [36]:
df.describe()

,text,intent
count,176,176
unique,172,26
top,Hola,NO_IDENTIFICADO
freq,3,35


In [37]:
df[df['text']=='Hola']

,text,intent
16,Hola,REGISTRAR_SALUDO
136,Hola,REGISTRAR_SALUDO
161,Hola,REGISTRAR_SALUDO


In [38]:
df['intent'].value_counts()

NO_IDENTIFICADO                   35
REGISTRAR_TOMA_MEDICAMENTO        28
REGISTRAR_ESTADO_EMOCIONAL        26
REGISTRAR_CITA_MEDICA             17
REGISTRAR_SALUDO                  11
REGISTRAR_DESPEDIDA                9
REGISTRAR_ALABANZA                 7
REGISTRAR_SITUACION_ADVERSA        6
REGISTRAR_DOMOTICA                 5
REGISTRAR_NECESIDAD                4
REGISTRAR_CONFIRMACION             4
REGISTRAR_ANECDOTA                 3
REGISTRAR_COVID                    3
CONSULTAR_ESTADO_PERSONA_MAYOR     3
REGISTRAR_MEDIDA_MEDICA            2
REGISTRAR_SINTOMA                  2
REGISTRAR_ACTIVIDAD                2
REGISTRAR_RECHAZO                  1
REGISTRAR_TELEGRAM                 1
CONSULTAR_TIEMPO                   1
ACTUALIZAR_TOMA_MEDICAMENTO        1
RECORDAR_CITA_MEDICA               1
RECORDAR_MEDICACION                1
REGISTRAR_DUDA                     1
ANOTAR_TOMA_MEDICAMENTO            1
CONSULTAR_NUTRICION                1
Name: intent, dtype: int64

In [39]:
dfFiltered = filterDataframe(df)
dfFiltered.head()

,text,intent
1,Hola estoy muy sola necesito de tus consejos y...,REGISTRAR_ESTADO_EMOCIONAL
2,Este mismo mes de junio a las 8'30 con el enfe...,REGISTRAR_CITA_MEDICA
3,"Hola, he estado un poco ocupado, lo siento, in...",REGISTRAR_ESTADO_EMOCIONAL
4,el rinialer me toca a las 9h,REGISTRAR_TOMA_MEDICAMENTO
6,Ahora voy a cenar luego hablamos 😊😊,REGISTRAR_DESPEDIDA


In [40]:
dfFiltered.shape

(91, 2)

In [41]:
dfFiltered['intent'].value_counts()

REGISTRAR_TOMA_MEDICAMENTO    28
REGISTRAR_ESTADO_EMOCIONAL    26
REGISTRAR_CITA_MEDICA         17
REGISTRAR_SALUDO              11
REGISTRAR_DESPEDIDA            9
Name: intent, dtype: int64

In [42]:
dfTrain, dfTest = saveTrainTestDataset(DATASET_VERSION)
DATASETS["v1"] = df
DATATRAIN["v1"] = dfTrain
DATATEST["v1"] = dfTest

Saved at /home/jovyan/data/dataset/v1/data_train.json
Saved at /home/jovyan/data/dataset/v1/data_test.json


El conjunto de datos no cumple los criterios de evaluación, ya que no llega a 100 registros, sino a 91 (y con registros repetidos), y hay una notable diferencia entre el número de registros de REGISTRAR_TOMA_MEDICAMENTO (28) y REGISTRAR_DESPEDIDA (9), por lo que el conjunto **no es válido**.

### 1.2. Conjunto v2

Tras la generación de nuevos diálogos por parte de los usuarios y la mejora de la herramienta que genera el conjunto de datos a partir de los diálogos para que elimine registros duplicados, se procede a analizar el nuevo conjunto de datos generado.

In [78]:
DATASET_VERSION = "v2" # Versión del conjunto de datos

df = loadDataframe(DATASET_VERSION)
df.head()

,text,intent
0,"Espero que no se enamore de ti como en Her, au...",NO_IDENTIFICADO
1,Hola estoy muy sola necesito de tus consejos y...,REGISTRAR_ESTADO_EMOCIONAL
2,Este mismo mes de junio a las 8'30 con el enfe...,REGISTRAR_CITA_MEDICA
3,Hola hoy Melo he pasado muy bien,REGISTRAR_ESTADO_EMOCIONAL
4,Adiós,REGISTRAR_DESPEDIDA


In [79]:
df.shape

(205, 2)

In [80]:
df["intent"].unique()

array(['NO_IDENTIFICADO', 'REGISTRAR_ESTADO_EMOCIONAL',
       'REGISTRAR_CITA_MEDICA', 'REGISTRAR_DESPEDIDA',
       'REGISTRAR_TOMA_MEDICAMENTO', 'REGISTRAR_NECESIDAD',
       'REGISTRAR_SALUDO', 'REGISTRAR_ALABANZA', 'REGISTRAR_CONFIRMACION',
       'CONSULTAR_ESTADO_PERSONA_MAYOR', 'REGISTRAR_COVID',
       'REGISTRAR_SITUACION_ADVERSA', 'ANOTAR_TOMA_MEDICAMENTO',
       'ACTUALIZAR_TOMA_MEDICAMENTO', 'REGISTRAR_MEDIDA_MEDICA',
       'REGISTRAR_SINTOMA', 'REGISTRAR_ANECDOTA', 'REGISTRAR_DUDA',
       'REGISTRAR_DOMOTICA', 'RECORDAR_MEDICACION',
       'RECORDAR_CITA_MEDICA', 'REGISTRAR_ACTIVIDAD', 'CONSULTAR_TIEMPO',
       'REGISTRAR_TELEGRAM', 'REGISTRAR_RECHAZO', 'CONSULTAR_NUTRICION'],
      dtype=object)

In [81]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    205 non-null    object
 1   intent  205 non-null    object
dtypes: object(2)
memory usage: 3.3+ KB


In [82]:
df.describe()

,text,intent
count,205,205
unique,199,26
top,Hola,NO_IDENTIFICADO
freq,4,40


In [83]:
df[df['text']=='Hola']

,text,intent
24,Hola,REGISTRAR_SALUDO
59,Hola,REGISTRAR_SALUDO
159,Hola,REGISTRAR_SALUDO
187,Hola,REGISTRAR_SALUDO


In [84]:
df['intent'].value_counts()

NO_IDENTIFICADO                   40
REGISTRAR_ESTADO_EMOCIONAL        29
REGISTRAR_TOMA_MEDICAMENTO        29
REGISTRAR_DESPEDIDA               20
REGISTRAR_CITA_MEDICA             19
REGISTRAR_SALUDO                  13
REGISTRAR_SITUACION_ADVERSA       10
REGISTRAR_ALABANZA                 7
REGISTRAR_DOMOTICA                 5
REGISTRAR_NECESIDAD                4
REGISTRAR_CONFIRMACION             4
REGISTRAR_ANECDOTA                 3
REGISTRAR_SINTOMA                  3
REGISTRAR_COVID                    3
CONSULTAR_ESTADO_PERSONA_MAYOR     3
REGISTRAR_MEDIDA_MEDICA            2
REGISTRAR_ACTIVIDAD                2
REGISTRAR_RECHAZO                  1
REGISTRAR_TELEGRAM                 1
CONSULTAR_TIEMPO                   1
ACTUALIZAR_TOMA_MEDICAMENTO        1
RECORDAR_CITA_MEDICA               1
RECORDAR_MEDICACION                1
REGISTRAR_DUDA                     1
ANOTAR_TOMA_MEDICAMENTO            1
CONSULTAR_NUTRICION                1
Name: intent, dtype: int64

In [85]:
filteredColumns = ['REGISTRAR_TOMA_MEDICAMENTO', 'REGISTRAR_ESTADO_EMOCIONAL', 'REGISTRAR_CITA_MEDICA', 'REGISTRAR_SALUDO', 'REGISTRAR_DESPEDIDA']
filter = df['intent'].isin(filteredColumns)

In [86]:
dfFiltered = filterDataframe(df)
dfFiltered.head()

,text,intent
1,Hola estoy muy sola necesito de tus consejos y...,REGISTRAR_ESTADO_EMOCIONAL
2,Este mismo mes de junio a las 8'30 con el enfe...,REGISTRAR_CITA_MEDICA
3,Hola hoy Melo he pasado muy bien,REGISTRAR_ESTADO_EMOCIONAL
4,Adiós,REGISTRAR_DESPEDIDA
5,"Hola, he estado un poco ocupado, lo siento, in...",REGISTRAR_ESTADO_EMOCIONAL


In [87]:
dfFiltered.shape

(110, 2)

In [88]:
dfFiltered['intent'].value_counts()

REGISTRAR_ESTADO_EMOCIONAL    29
REGISTRAR_TOMA_MEDICAMENTO    29
REGISTRAR_DESPEDIDA           20
REGISTRAR_CITA_MEDICA         19
REGISTRAR_SALUDO              13
Name: intent, dtype: int64

In [89]:
dfTrain, dfTest = saveTrainTestDataset(DATASET_VERSION)
DATASETS["v2"] = df
DATATRAIN["v2"] = dfTrain
DATATEST["v2"] = dfTest

Saved at /home/jovyan/data/dataset/v2/data_train.json
Saved at /home/jovyan/data/dataset/v2/data_test.json


El conjunto de datos cumple los criterios de evaluación, supera los 100 registros y está balanceado, por lo tanto, **es válido**.

## 2. Entrenamiento de modelos

In [94]:
def getValidator(languajeModel):
    if languajeModel == "dialogflow":
        return identifyIntentWithDialogflow

def validate(test, languajeModel):
    validator = getValidator(languajeModel)
    total = 0
    success = 0
    for item in test:
        expectedIntent = item["intent"]
        # En Google Action, el intent NO_IDENTIFICADO es el intent por defecto, que 
        # conservamos con el nombre que Google le da, por legibilidad.
        if "NO_IDENTIFICADO" == expectedIntent:
            expectedIntent = "Default Fallback Intent"
        text = item["text"]
        receivedIntent = validator(text)
        if expectedIntent == receivedIntent:
            success += 1
        else:
            print(f"FAILED: expected: {expectedIntent}, received: {receivedIntent} on text: {text}")
        total += 1
    accuracy = round(success/total*100, 2)
    return total, success, accuracy

In [95]:
import os
from google.cloud import dialogflow

PROJECT_ID = "***REMOVED***"
SESSION = "123456789"
LANGUAGE_CODE = "es"

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/jovyan/credentials/google/key.json"

def identifyIntentWithDialogflow(text):
    session_client = dialogflow.SessionsClient()
    session = session_client.session_path(PROJECT_ID, SESSION)
    text_input = dialogflow.TextInput(text=text, language_code=LANGUAGE_CODE)
    query_input = dialogflow.QueryInput(text=text_input)
    response = session_client.detect_intent(
        request={"session": session, "query_input": query_input}
    )
    intent = response.query_result.intent.display_name
    return intent


### 2.1. Modelo v1: Datasetv2 - Dialogflow

In [96]:
DATASET_VERSION = "v2" # Versión del conjunto de datos
LANGUAJE_MODEL = "dialogflow"

**WARNING:** Generar manualmente el conjunto de entrenamiento de Dialogflow desde la consola:

```
python cli_dialogflow_generate.py v2 v1
```

A continuación:

- Eliminar todos los intents del proyecto de Dialogflow, excepto el "DefaultFallbackIntent".
- Importar en el proyecto de Dialogflow el dataset generado, comprimido en .zip (no añadir model_info.json).
- Eliminar el intent "NO_IDENTIFICADO", puesto que contiene textos que, por ahora, no han sido clasificados, de modo que deberían ir a "DefaultFallbackIntent".

In [97]:
test = getDataset(DATASET_VERSION, "_test")
total, success, accuracy = validate(test, LANGUAJE_MODEL)

print(f"Total: {total}, success: {success}, accuracy: {accuracy}%")

FailedPrecondition: 400 Intent with id 'fb55994e-c4de-4855-b6a1-704cd55beb32' not found among intents in environment '' for agent with id 'f81ec4ad-9422-4515-9d52-9fecdb8a8cd9'.
com.google.apps.framework.request.StatusException: <eye3 title='FAILED_PRECONDITION'/> generic::FAILED_PRECONDITION: Intent with id 'fb55994e-c4de-4855-b6a1-704cd55beb32' not found among intents in environment '' for agent with id 'f81ec4ad-9422-4515-9d52-9fecdb8a8cd9'.

El modelo **no es válido**, puesto que el criterio de evaluación exige una tasa de acierto superior al 85%, y este modelo ha alcanzado un 82.93%.